In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install datasets==2.18.0 transformers[torch]==4.38.2 tensorboard evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset, load_metric
import numpy as np
import torch
from huggingface_hub import notebook_login
import evaluate

## Load Model

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
# lug_Latn actually already exists as a language code
tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="eng_Latn", target_lang="lug_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Just a test sentence

In [ ]:
article = "All refugees were requested to register with the chairman."
inputs = tokenizer(article, return_tensors="pt")

In [ ]:
translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["lug_Latn"],
    max_length=360,
)
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


"Abanoonyi b'obubudamu bonna basabiddwa okwewandiisa mu ssentebe."

# Freeze some layers to make the model smaller
- I will experiment by freezing the entire encoder, which is less task-specific compared to the decoder (in translation)

In [ ]:
for param in model.model.encoder.parameters():
    param.requires_grad = False

for name, param in model.model.encoder.named_parameters():
    if param.requires_grad:
        print(f"{name} is trainable")
    else:
        print(f"{name} is frozen")

embed_tokens.weight is frozen
layers.0.self_attn.k_proj.weight is frozen
layers.0.self_attn.k_proj.bias is frozen
layers.0.self_attn.v_proj.weight is frozen
layers.0.self_attn.v_proj.bias is frozen
layers.0.self_attn.q_proj.weight is frozen
layers.0.self_attn.q_proj.bias is frozen
layers.0.self_attn.out_proj.weight is frozen
layers.0.self_attn.out_proj.bias is frozen
layers.0.self_attn_layer_norm.weight is frozen
layers.0.self_attn_layer_norm.bias is frozen
layers.0.fc1.weight is frozen
layers.0.fc1.bias is frozen
layers.0.fc2.weight is frozen
layers.0.fc2.bias is frozen
layers.0.final_layer_norm.weight is frozen
layers.0.final_layer_norm.bias is frozen
layers.1.self_attn.k_proj.weight is frozen
layers.1.self_attn.k_proj.bias is frozen
layers.1.self_attn.v_proj.weight is frozen
layers.1.self_attn.v_proj.bias is frozen
layers.1.self_attn.q_proj.weight is frozen
layers.1.self_attn.q_proj.bias is frozen
layers.1.self_attn.out_proj.weight is frozen
layers.1.self_attn.out_proj.bias is froze

## Load Data

In [ ]:
df = pd.read_csv("gdrive/My Drive/nllb-finetuning/English_Luganda_Corpus.csv", encoding="latin-1")
print(df.shape)
print(df.columns)
df.head()

(15022, 5)
Index(['Lemmatized US English', 'US English', 'Luganda', 'Unnamed: 3',
       'Unnamed: 4'],
      dtype='object')


,Lemmatized US English,US English,Luganda,Unnamed: 3,Unnamed: 4
0,All refugees were requested to register with t...,All refugees were requested to register with t...,Abanoonyiboobubudamu bonna baasabiddwa beewand...,NaN,NaN
1,They called for a refugees' meeting yesterday.,They called for a refugees' meeting yesterday.,Baayise olukungaana lw'abanoonyiboobubudamu eg...,NaN,NaN
2,Refugees had misunderstandings between thems...,Refugees had misunderstandings between thems...,Abanoonyiboobubudamu b'abadde n'obutakkaanya w...,NaN,NaN
3,We were urged to welcome refugees into our com...,We were urged to welcome refugees into our com...,Twakubirizibwa okwaniriza abanoonyiboobubudamu...,NaN,NaN
4,More development is achieved when we work toge...,More development is achieved when we work toge...,Bwe tukolera awamu enkulaakulana enyingi efuni...,NaN,NaN


In [ ]:
# since df has 2 unnamed columns, let's remove them
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.drop(df.columns[df.columns.str.contains('Lemmatized US English')], axis = 1, inplace = True)
df.head()

,US English,Luganda
0,All refugees were requested to register with t...,Abanoonyiboobubudamu bonna baasabiddwa beewand...
1,They called for a refugees' meeting yesterday.,Baayise olukungaana lw'abanoonyiboobubudamu eg...
2,Refugees had misunderstandings between thems...,Abanoonyiboobubudamu b'abadde n'obutakkaanya w...
3,We were urged to welcome refugees into our com...,Twakubirizibwa okwaniriza abanoonyiboobubudamu...
4,More development is achieved when we work toge...,Bwe tukolera awamu enkulaakulana enyingi efuni...


In [ ]:
print(df.isna().sum())

Lemmatized US English    0
US English               0
Luganda                  1
dtype: int64


In [ ]:
df.dropna(inplace=True)
print(df.isna().sum())

Lemmatized US English    0
US English               0
Luganda                  0
dtype: int64


## Training

In [ ]:
def tokenize_and_create_dataset(tokenizer, df):
    # tokenizer() returns a dictionary w keys: input_ids (numerical representation of tokens), attention_mask
    encodings = tokenizer(
        list(df["US English"]),
        padding=True,
        truncation=True
    )

    decodings = tokenizer(
        list(df["Luganda"]),
        padding=True,
        truncation=True
    )

    dataset = Dataset.from_dict({
        "input_ids": encodings["input_ids"],
        "attention_mask": encodings["attention_mask"],
        "labels": decodings["input_ids"]
    })
    return dataset

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels] #turns into list of lists to be suitable for evaluation like bleu
    return preds, labels

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True) # decodes the logits to get predicted tokens
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    bleu = evaluate.load("bleu")
    result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["bleu"]}
    return result


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_dataset = tokenize_and_create_dataset(tokenizer, train_df)
test_dataset = tokenize_and_create_dataset(tokenizer, test_df)
print('Data has been tokenized')

Data has been tokenized


In [ ]:
print(train_dataset)
print(train_dataset[0])

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 12016
})
{'input_ids': [256047, 64049, 69796, 2442, 205680, 108, 8334, 20362, 30330, 202, 17334, 8334, 107024, 462, 5346, 248075, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [256047, 1147, 2395, 8078, 161, 214656, 2651, 26346, 77, 1137, 93273, 211525, 94565, 41002, 133918, 183239, 60798, 23966, 1017, 115, 22978, 107540, 248075, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
NUM_EPOCHS = 2
model_args = Seq2SeqTrainingArguments(
    output_dir="gdrive/My Drive/nllb-finetuning",
    weight_decay = 0.02,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    evaluation_strategy="steps",
    save_strategy="no",
    load_best_model_at_end=True,
    logging_strategy="steps",
    learning_rate=2e-4,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=model_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [29]:
trainer.train()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Bleu
500,5.953000,5.061834,0.200466
1000,5.098000,4.556126,0.218957
1500,4.662100,4.230774,0.236111
2000,4.314600,4.035874,0.245302
2500,4.142500,3.915684,0.256474


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation 

Step,Training Loss,Validation Loss,Bleu
500,5.953000,5.061834,0.200466
1000,5.098000,4.556126,0.218957
1500,4.662100,4.230774,0.236111
2000,4.314600,4.035874,0.245302
2500,4.142500,3.915684,0.256474
3000,4.059900,3.874373,0.259220


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3004, training_loss=4.704081055644984, metrics={'train_runtime': 4094.411, 'train_samples_per_second': 5.869, 'train_steps_per_second': 0.734, 'total_flos': 2746398751064064.0, 'train_loss': 4.704081055644984, 'epoch': 2.0})